In [8]:
# refs: https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/

In [14]:
from dotenv import load_dotenv
import os
import sys
import logging

from llama_index.core import StorageContext, load_index_from_storage

from llama_index.core.retrievers import VectorIndexRetriever

from llama_index.core import Settings

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

from llama_index.core.data_structs import Node
from llama_index.core.schema import NodeWithScore
from llama_index.core import get_response_synthesizer


import openai

import nest_asyncio
nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
from llama_index.postprocessor.rankgpt_rerank import RankGPTRerank

import pandas as pd
from IPython.display import display, HTML


load_dotenv()  # Load environment variables from .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

OPENAI_API_KEY = openai.api_key

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


%load_ext autoreload
%autoreload 2




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:


# Rebuild the storage context
storage_context = StorageContext.from_defaults(persist_dir="./persist")

# Load the index from storage
index = load_index_from_storage(storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [16]:

def get_retrieved_nodes(
    query_str, vector_top_k=10, reranker_top_n=3, with_reranker=False
):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = RankGPTRerank(
            llm=OpenAI(
               model="gpt-3.5-turbo",
               temperature=0.0,
               api_key=OPENAI_API_KEY,
            ),
            top_n=reranker_top_n,
            verbose=True,
        )
        retrieved_nodes = reranker.postprocess_nodes(
            retrieved_nodes, query_bundle
        )

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

In [17]:
query = "What are potential CONSECUENCES of keytruda?"
retrieved_nodes = get_retrieved_nodes(
    query,
    vector_top_k=30,
    reranker_top_n=5,
    with_reranker=True,
)

# for idx, node in enumerate(retrieved_nodes):
#     print(f"Retrieved Node {idx}: Embedding:  {node.embedding}, embedding_VDB: {index.vector_store.get(node.id_)}")

visualize_retrieved_nodes(retrieved_nodes)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completion

,Score,Text
0,0.694664,"Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as colitis, hepatitis, and pneumonitis."
1,0.630779,How does Keytruda affect immune system function in NSCLC patients? Keytruda enhances the immune system's ability to detect and destroy cancer cells but can also lead to immune-related adverse effects that need to be managed.
2,0.655395,"Are there specific side effects of Keytruda that NSCLC patients should monitor? NSCLC patients should monitor for cough, shortness of breath, and chest pain, as these could indicate immune-related pneumonitis."
3,0.632893,Can Keytruda be used in NSCLC patients with autoimmune diseases? Keytruda should be used with caution in patients with pre-existing autoimmune diseases due to the risk of exacerbating the condition.
4,0.602679,"Are there any specific monitoring protocols for NSCLC patients on Keytruda as suggested by KEYNOTE-006 outcomes? Regular monitoring of liver function, lung function, and immune markers is recommended to manage potential side effects effectively."


In [18]:

response_synthesizer = get_response_synthesizer(response_mode="compact")

response = response_synthesizer.synthesize(
    query, nodes=retrieved_nodes
)
response

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response(response='Potential consequences of Keytruda include immune-related adverse effects such as colitis, hepatitis, pneumonitis, and the exacerbation of pre-existing autoimmune diseases. Regular monitoring of liver function, lung function, and immune markers is recommended to manage these potential side effects effectively.', source_nodes=[NodeWithScore(node=TextNode(id_='1c86f3e8-53f4-417a-9c1b-81fb42255e97', embedding=None, metadata={'document_title': 'Keytruda and Immune-Related Adverse Effects: A Comprehensive Overview'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='49de588d-beb9-4e7e-84dc-06ffb2e63a2d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c73be1b4e2cb5e6157fd80000f0fe59f6ca8631fa4924c5ed0be7dd3180c3218')}, text='Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as colitis, hepatitis, and pneumonitis.', mim

In [ ]:
from llama_index.core import ResponseSynthesizer

# Assuming 'retrieved_nodes' is already obtained from your previous function
query = "What are potential CONSEQUENCES of keytruda?"

# Create a response synthesizer
response_synthesizer = ResponseSynthesizer.from_args()

# Synthesize a response using the retrieved nodes and your query
response = response_synthesizer.synthesize(
    query=query,
    nodes=retrieved_nodes
)

# Display the generated answer
print(response)